# SakamoBERTo

Fine-tune a Japanese BERT model on a Sakamoto tweet classification task.

## Install Transformers

In [1]:
!git clone https://github.com/huggingface/transformers
!cd transformers;pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 26433 (delta 11), reused 19 (delta 7), pack-reused 26405
Receiving objects: 100% (26433/26433), 15.87 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (18417/18417), done.
Processing /content/transformers
     |████████████████████████████████| 3.8MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 39.4MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 
  Created wheel for transformers: filename=transformers-2.9.1-cp36-none-any.whl size=637722 sha256=a31612578b8d41a5a8a9ac5738f84313d09abfd28efe566162470a63bb50c45c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dcu25m22/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=37a695edbfa7786744e0aebb7eda15e438bf3f9ad706ff2c1846e3fb63850de5

## Download CoLA data

In [0]:
# !wget https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py
# !python download_glue_data.py --data_dir glue_data --tasks all

In [0]:
!cp -r drive/My\ Drive/glue_data .

## Download a Japanese model

In [4]:
!wget http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip
!unzip -d bert Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip

--2020-05-16 05:07:38--  http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip
Resolving nlp.ist.i.kyoto-u.ac.jp (nlp.ist.i.kyoto-u.ac.jp)... 133.3.252.171
Connecting to nlp.ist.i.kyoto-u.ac.jp (nlp.ist.i.kyoto-u.ac.jp)|133.3.252.171|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 411573567 (393M) [application/zip]
Saving to: ‘Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip’

Japanese_L-12_H-768 100%[===================>] 392.51M   628KB/s    in 10m 52s 

2020-05-16 05:18:30 (617 KB/s) - ‘Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip’ saved [411573567/411573567]

Archive:  Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip
   creating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/
  inflating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json  
  inflating: bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/pytorch_model.bin  
  inflating: bert/Japanese_L-12_H-768_

## Fine-tune the model

In [5]:
!pip install -U future
!cd transformers;pip install -r ./examples/requirements.txt

     |████████████████████████████████| 829kB 4.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=ecd169c642f8cd9cb815c3f7cbec2ca9e0a100b2ba1b5b238590fa628026e645
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 204kB 13.6MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=05da42ff86bda753dad2bc7e45e57855b699699ee76f9f6f2b4258221f9e51d1
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [6]:
!python transformers/examples/text-classification/run_glue.py \
    --model_type bert \
    --model_name_or_path bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers \
    --task_name CoLA \
    --do_train \
    --do_eval \
    --data_dir glue_data/CoLA \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=8   \
    --per_gpu_train_batch_size=8   \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir sakamoto_bert

2020-05-16 05:18:51.647441: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/16/2020 05:18:53 - INFO - transformers.training_args -   PyTorch: setting up devices
05/16/2020 05:18:53 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/16/2020 05:18:53 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='sakamoto_bert', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=2e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_co

In [0]:
# !cp -r sakamoto_bert drive/My\ Drive

## Do a sequence classification

In [0]:
import torch

In [0]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('sakamoto_bert')
model = AutoModelForSequenceClassification.from_pretrained('sakamoto_bert')

In [0]:
sequences = []
with open('glue_data/CoLA/test.tsv') as f:
  for line in f:
    _, sequence = line.strip().split('\t')
    sequences.append(sequence)

In [11]:
classes = ['aozam3', 'sksk_sskn']

for sequence in sequences:
    sequence_tensor = tokenizer.encode(sequence, return_tensors="pt")
    classification_logits = model(sequence_tensor)[0]
    
    results = torch.softmax(classification_logits, dim=1).tolist()[0]

    print(sequence)
    for i in range(2):
        print('{}: {}'.format(classes[i], results[i]))
    print()

母 に ハンネ で 呼ば れた ので スマホ 食った
aozam3: 0.979850172996521
sksk_sskn: 0.020149847492575645

外 寒 すぎて デッケェ 冷え ピタ に 包ま れた とき と 全く 同じ 気持ち に なった
aozam3: 0.0017393020680174232
sksk_sskn: 0.9982606768608093

密度 が 高 すぎる ため 少量で 胃 の 質量 を 5000 倍 に し 人間 を 内側 から 破壊 する 餅 と いう 食材 、 本当に 面白い
aozam3: 0.006239396519958973
sksk_sskn: 0.9937606453895569

君 は 私 の 手 に は 負え ない よ …… トホホ …… と 思わ せて くれる 担当 、 好きです
aozam3: 0.0006770329200662673
sksk_sskn: 0.9993230104446411

量 を 重要 視 した ” 食 ” を 行う に あたって 障害 と さ れる 熱 と 冷え は 取り除く と いい らしい です ワザップ に よる と
aozam3: 0.0015453067608177662
sksk_sskn: 0.9984546899795532



# References

- Transformers
    - https://github.com/huggingface/transformers
    - https://huggingface.co/transformers/index.html
    - https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/huggingface_pytorch-transformers.ipynb

- CoLA
    - https://nyu-mll.github.io/CoLA/

- BERT Japanese pretrained model
    - http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT%E6%97%A5%E6%9C%AC%E8%AA%9EPretrained%E3%83%A2%E3%83%87%E3%83%AB


- Qiita
    - https://qiita.com/neonsk/items/27424d6122e00fe632b0
    - https://qiita.com/nekoumei/items/7b911c61324f16c43e7e
    - https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3
    - https://qiita.com/knok/items/9e3b4505d6b8f813943d